<div>
    <p style="float: right;"><img width="66%" src="templates/logo_fmriflows.gif"></p>
    <h1>Functional Preprocessing</h1>
    <p>This notebooks preprocesses functional MRI images by executing the following processing steps:

1. Image preparation
    1. Reorient images to RAS
    1. Removal of non-steady state volumes 
    1. Brain extraction with Nilearn
1. Motion correction
    1. Either direct motion correction with FSL
    1. Or, if low-pass filter specified, multistep motion correction with FSL and Python
1. Slice-wise correction with SPM
1. Two-step coregistration using Rigid and BBR with FSL, using WM segmentation from SPM
1. Temporal filtering with AFNI (optional)
1. Spatial filtering (i.e. smoothing) with Nilearn

Additional, this workflow also computes:
 - Friston's 24-paramter model for motion parameters
 - Framewise Displacement (FD) and DVARS
 - Average signal in total volume, in GM, in WM and in CSF
 - Anatomical CompCor components
 - Temporal CompCor components
 - Independent components in image before smoothing
 
**Note:** This notebook requires that the anatomical preprocessing pipeline was already executed and that it's output can be found in the dataset folder under `dataset/derivatives/fmriflows/preproc_anat`. </p>
</div>

## Data Structure Requirements

The data structure to run this notebook should be according to the BIDS format:

    dataset
    ├── fmriflows_spec_preproc.json
    ├── sub-{sub_id}
    │   └── func
    │       └── sub-{sub_id}_task-{task_id}[_run-{run_id}]_bold.nii.gz
    └── task-{task_id}_bold.json
    
**Note:** Subfolders for individual scan sessions and `run` identifiers are optional.

`fmriflows` will run the preprocessing on all files of a particular subject and a particular task.

## Execution Specifications

This notebook will extract the relevant processing specifications from the `fmriflows_spec_preproc.json` file in the dataset folder. In the current setup, they are as follows:

In [ ]:
import json
from os.path import join as opj

spec_file = opj('/data', 'fmriflows_spec_preproc.json')

with open(spec_file) as f:
    specs = json.load(f)

In [ ]:
# Extract parameters for functional preprocessing workflow
subject_list = specs['subject_list_func']
session_list = specs['session_list_func']
task_list = specs['task_list']
run_list = specs['run_list']
ref_timepoint = specs['ref_timepoint']
res_func = specs['res_func']
filters_spatial = specs['filters_spatial']
filters_temporal = specs['filters_temporal']
n_compcor_confounds = specs['n_compcor_confounds']
outlier_thr = specs['outlier_thresholds']
n_independent_components = specs['n_independent_components']
n_proc = specs['n_parallel_jobs']

If you'd like to change any of those values manually, overwrite them below:

In [ ]:
# List of subject identifiers
subject_list

In [ ]:
# List of session identifiers
session_list

In [ ]:
# List of task identifiers
task_list

In [ ]:
# List of run identifiers
run_list

In [ ]:
# Reference timepoint for slice time correction (in ms)
ref_timepoint

In [ ]:
# Requested voxel resolution after coregistration of functional images
res_func

In [ ]:
# List of spatial filters (smoothing) to apply (separetely, i.e. with iterables)
# Values are given in mm
filters_spatial

In [ ]:
# List of temporal filters to apply (separetely, i.e. with iterables)
# Values are given in seconds
filters_temporal

In [ ]:
# Number of CompCor components to compute
n_compcor_confounds

In [ ]:
# Threshold for outlier detection (3.27 represents a threshold of 99.9%)
# Values stand for FD, DVARS, TV, GM, WM, CSF
outlier_thr

In [ ]:
# Number of independent components to compute
n_independent_components

In [ ]:
# Number of parallel jobs to run
n_proc

In [ ]:
res_norm = [2.0, 2.0, 2.0]
norm_func = True

# Creating the Workflow

To ensure a good overview of the functional preprocessing, the workflow was divided into three subworkflows:

1. The Main Workflow, i.e. doing the actual preprocessing. Containing subworkflows for...
    1. Image preparation
    1. Motion correction
    1. Image coregistration
    1. Temporal filtering (optional)
2. The Confound Workflow, i.e. computing confound variables
3. Report Workflow, i.e. visualizating relevant steps for quality control

## Import Modules

In [ ]:
import os
import numpy as np
from os.path import join as opj
from nipype import Workflow, Node, IdentityInterface, Function
from nipype.interfaces.image import Reorient
from nipype.interfaces.fsl import FLIRT
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.confounds import (
    ACompCor, TCompCor, FramewiseDisplacement, ComputeDVARS)

In [ ]:
# Specify SPM location
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('/opt/spm12-r7219/spm12_mcr/spm12')

## Relevant Execution Variables

In [ ]:
# Folder paths and names
exp_dir = '/data/derivatives'
out_dir = 'fmriflows'
work_dir = '/workingdir'

## Create a subworkflow for the Main Workflow

### Image preparation subworkflow

In [ ]:
# Reorient anatomical images to RAS
reorient = Node(Reorient(orientation='RAS'), name='reorient')

In [ ]:
# Extract brain from functional image
def extract_brain(in_file):

    from nipype.interfaces.fsl import BET
    from nipype.interfaces.ants import N4BiasFieldCorrection
    from nilearn.image import mean_img, new_img_like, load_img
    from scipy.ndimage import binary_dilation, binary_fill_holes
    from os.path import basename, abspath

    # Compute mean image
    img_mean = mean_img(in_file).to_filename('mean.nii.gz')

    # Apply N4BiasFieldCorrection on mean file
    res = N4BiasFieldCorrection(input_image='mean.nii.gz',
                                dimension=3, copy_header=True).run()

    # Create brain mask based on functional bias corrected mean file
    res = BET(in_file=res.outputs.output_image, mask=True,
              no_output=True, robust=True).run()
    
    # Dilate mask and fill holes
    img_mask = load_img(res.outputs.mask_file)
    mask = binary_fill_holes(binary_dilation(img_mask.get_data(),
                                             iterations=2))
    img_mask = new_img_like(in_file, mask, copy_header=True)

    # Save mask image
    mask_file = abspath(basename(in_file).replace('.nii', '_mask.nii'))
    img_mask.to_filename(mask_file)
    
    return mask_file

mask_func_brain = Node(Function(input_names=['in_file'],
                                output_names=['mask_file'],
                                function=extract_brain),
                       name='mask_func_brain')

In [ ]:
# Detect Non-Steady State volumes and save information to file
def detect_non_stead_states(in_file):
    
    import numpy as np
    from os.path import basename, abspath
    from nipype.algorithms.confounds import NonSteadyStateDetector
    
    # Detect Non-Steady State volumes
    res = NonSteadyStateDetector(in_file=in_file).run()
    t_min = res.outputs.n_volumes_to_discard
    
    nss_file = abspath(basename(in_file).replace('.nii.gz', '_nss.txt'))
    np.savetxt(nss_file, [t_min], fmt='%d')
    
    return t_min, nss_file

nss_detection = Node(Function(input_names=['in_file'],
                              output_names=['t_min', 'nss_file'],
                              function=detect_non_stead_states),
                     name='nss_detection')

In [ ]:
# Create image preparation workflow
prepareflow = Workflow(name='prepareflow')

# Add nodes to workflow and connect them
prepareflow.connect([(reorient, nss_detection, [('out_file', 'in_file')]),
                     (reorient, mask_func_brain, [('out_file', 'in_file')]),
                    ])

### Motion & Slice-time correction nodes

In [ ]:
# Remove NSS volumes and estimate original motion parameters on masked brain
def estimate_motion_parameters(in_file, mask_file, t_min):

    import os
    from nipype.interfaces.fsl import MCFLIRT
    from nilearn.image import load_img, math_img, new_img_like
    from os.path import basename, abspath, dirname

    # Specify name of output file
    out_file = abspath(basename(in_file).replace('.nii.gz', '_mcf.nii.gz'))

    # Remove NSS volumes from functional image
    img = load_img(in_file).slicer[..., t_min:]

    # Apply brain mask to functional image, reset header and save file as NIfTI
    img_clean = math_img('img * mask[..., None]', img=img, mask=mask_file)
    img_clean = new_img_like(img, img_clean.get_data(), copy_header=True)
    img_clean.to_filename(out_file)

    # Performe initial motion correction
    res = MCFLIRT(mean_vol=True,
                  save_plots=True,
                  output_type='NIFTI',
                  save_mats=True,
                  in_file=out_file,
                  out_file=out_file).run()

    # Remove mcf file to save space
    os.remove(out_file)
        
    # Aggregate outputs
    outputs = [res.outputs.mean_img,
               res.outputs.par_file,
               dirname(res.outputs.mat_file[0])]
    
    return outputs

estimate_motion = Node(Function(input_names=['in_file', 'mask_file', 't_min'],
                                  output_names=['mean_file', 'par_file', 'mat_folder'],
                                  function=estimate_motion_parameters),
                         name='estimate_motion')

In [ ]:
# Apply low-pass filters to motion parameters and prepare MAT-files
def filter_motion_parameters(mean_file, par_file, mat_folder, tFilter, TR):

    import os
    import numpy as np
    from glob import glob
    from math import cos, sin
    from scipy.signal import butter, filtfilt
    from os.path import basename, abspath, exists
    import subprocess
    import warnings

    # Specify name of output file
    out_file = abspath(basename(par_file))
    
    # Collect MAT files
    mat_file = sorted(glob('%s/MAT_????' % mat_folder))
    new_mats = abspath('mats_files')
    
    # Function to low-pass filter FSL motion parameters
    def clean_par(pars, TR, low_pass):

        # Taken from nilearn.signal
        def _check_wn(freq, nyq):
            wn = freq / float(nyq)
            if wn >= 1.:
                wn = 1 - 10 * np.finfo(1.).eps
                warnings.warn(
                    'The frequency specified for the low pass filter is '
                    'too high to be handled by a digital filter (superior to '
                    'nyquist frequency). It has been lowered to %.2f (nyquist '
                    'frequency).' % wn)
            if wn < 0.0: # equal to 0.0 is okay
                wn = np.finfo(1.).eps
                warnings.warn(
                    'The frequency specified for the low pass filter is too low'
                    ' to be handled by a digital filter (must be non-negative).'
                    ' It has been set to eps: %.5e' % wn)
            return wn    

        # Taken from nilearn.signal
        def butterworth(signals, sampling_rate, low_pass, order=5):
            nyq = sampling_rate * 0.5
            critical_freq = _check_wn(low_pass, nyq)
            b, a = butter(order, critical_freq, 'low', output='ba')
            signals = filtfilt(b, a, signals, axis=0)
            return signals

        # Filter signal
        pars_clean = butterworth(pars, 1./TR, low_pass)

        return pars_clean


    # Function to compute affine rotation matrix based on FSL rotation angles
    def rot_mat(theta):

        R_x = np.array([[1, 0,             0],
                        [0, cos(theta[0]), sin(theta[0])],
                        [0,-sin(theta[0]), cos(theta[0])]])

        R_y = np.array([[cos(theta[1]), 0,-sin(theta[1])],
                        [0,             1, 0],
                        [sin(theta[1]), 0, cos(theta[1])]])

        R_z = np.array([[ cos(theta[2]), sin(theta[2]), 0],
                        [-sin(theta[2]), cos(theta[2]), 0],
                        [ 0,             0,             1]])

        return np.dot(R_z, np.dot(R_y, R_x))
    
    
    # Perform second motion correction with low-pass filter if specified
    if tFilter[0]:

        # Extract low-pass filter value
        low_pass = 1. / tFilter[0]

        # Low-pass filter rotation angles
        radi = np.loadtxt(par_file)[:, :3]
        clean_radi = clean_par(radi, TR, low_pass)

        #Extract translation parameters from FSL's MAT files
        trans = []
        for m in mat_file:
            M = np.loadtxt(m)
            R = M[:3,:3]

            # Back-project translation parameters into origin space
            trans.append(np.array(np.dot(np.linalg.inv(R), M[:3, -1])))

        trans_o = np.array(trans)

        # Low-pass filter translation parameters
        clean_trans_o = clean_par(trans_o, TR, low_pass)
        
        # Create output folder for new MAT files
        if not exists(new_mats):
            os.makedirs(new_mats)

        # Forward-project translation parameter into FSL space and save them
        mat_files = []
        clean_trans = []
        for i, p in enumerate(clean_trans_o):
            R = rot_mat(clean_radi[i])
            tp = np.array(np.dot(R, clean_trans_o[i]))
            clean_trans.append(tp)
            mat = np.vstack((np.hstack((R, tp[..., None])), [0,0,0,1]))
            new_mat_path = '%s/MAT_%04d' % (new_mats, i)
            mat_files.append(new_mat_path)
            np.savetxt(fname=new_mat_path, X=mat, delimiter=" ", fmt='%.6f')
            
        # Overwrite FSL's pars file with new parameters
        new_radi = []
        new_trans = []

        for m in mat_files:
            cmd = 'avscale --allparams %s %s' % (m, mean_file)
            process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
            pout = process.communicate()[0].decode("utf-8").split('\n')
            for p in pout:
                if 'Rotation Angles (x,y,z)' in p:
                    new_radi.append(np.array(p[32:].split(), dtype='float'))
                if 'Translations (x,y,z)' in p:
                    new_trans.append(np.array(p[27:].split(), dtype='float'))
        new_pars = np.hstack((new_radi, new_trans))
        np.savetxt(out_file, new_pars, fmt='%.8e')
    
    else:
        
        out_file = abspath(basename(par_file))
        np.savetxt(out_file, np.loadtxt(par_file), fmt='%.8e')
        new_mats = mat_folder

    return out_file, new_mats

motion_parameters = Node(Function(input_names=['mean_file', 'par_file', 'mat_folder',
                                               'tFilter', 'TR'],
                                  output_names=['par_file', 'mat_folder'],
                                  function=filter_motion_parameters),
                         name='motion_parameters')

In [ ]:
# Correct for slice-wise acquisition
def correct_for_slice_time(in_files, TR, slice_order, nslices,
                           time_acquisition, ref_timepoint):
    
    import os
    import numpy as np
    from nilearn.image import load_img, new_img_like
    from nipype.interfaces.spm import SliceTiming
    from os.path import basename, abspath

    # Check if slice-time correction need to be performed or not
    if len(np.unique(slice_order)) == 1:
        timecorrected_files = in_files
    else:
    
        # Specify name of output file and decompress it for SPM
        out_file = abspath(basename(in_files).replace('.nii.gz', '_stc.nii'))
        load_img(in_files).to_filename(out_file)

        # Perform slice time correction
        res = SliceTiming(in_files=out_file,
                          ref_slice=ref_timepoint,
                          time_repetition=TR,
                          slice_order=slice_order,
                          num_slices=nslices,
                          time_acquisition=time_acquisition).run()
        os.remove(out_file)
        stc_file = res.outputs.timecorrected_files

        # Reset TR value in header and compress output to reduce file size
        timecorrected_files = stc_file.replace('.nii', '.nii.gz')
        img_out = load_img(stc_file)
        img_out = new_img_like(in_files, img_out.get_data(), copy_header=True)
        img_out.header.set_zooms(list(img_out.header.get_zooms()[:3]) + [TR])
        img_out.to_filename(timecorrected_files)
        os.remove(stc_file)
        
    return timecorrected_files

slice_time = Node(Function(input_names=['in_files', 'TR', 'slice_order', 'nslices',
                                        'time_acquisition', 'ref_timepoint'],
                              output_names=['timecorrected_files'],
                              function=correct_for_slice_time),
                     name='slice_time')
slice_time.inputs.ref_timepoint = ref_timepoint

In [ ]:
# Apply warp Motion Correction, Coregistration (and Normalization)
def apply_warps(in_file, mat_folder, coreg, brain, transforms,
                template, norm_func, t_min, TR):
    
    import os
    import numpy as np
    from glob import glob
    from os.path import basename, abspath
    from nipype.interfaces.ants import ApplyTransforms
    from nipype.interfaces.c3 import C3dAffineTool
    from nilearn.image import (iter_img, load_img, mean_img, concat_imgs,
                               new_img_like, resample_to_img, threshold_img)
    
    # Specify name of output file and decompress it for SPM
    out_file = abspath(basename(in_file.replace('.nii', '_warped.nii')))
    if norm_func:
        reference = template
    else:
        reference = 'temp_func.nii.gz'

    # Apply warp for each volume individually
    out_list = []
    mat_files = sorted(glob(mat_folder + '/MAT_????'))

    # Remove NSS volumes from functional image
    img = load_img(in_file).slicer[..., t_min:]
    
    for i, e in enumerate(iter_img(img)):

        temp_file = 'temp_func.nii.gz'
        e.to_filename(temp_file)

        c3d_coreg = C3dAffineTool(fsl2ras=True,
                                  transform_file=coreg,
                                  source_file='temp_func.nii.gz',
                                  reference_file=brain,
                                  itk_transform='temp_coreg.txt').run()

        c3d_mc = C3dAffineTool(fsl2ras=True,
                               transform_file=mat_files[i],
                               source_file='temp_func.nii.gz',
                               reference_file='temp_func.nii.gz',
                               itk_transform='temp_mats.txt').run()

        if norm_func:
            transform_list = [transforms,
                              c3d_coreg.outputs.itk_transform,
                              c3d_mc.outputs.itk_transform]
        else:
            transform_list = [c3d_coreg.outputs.itk_transform,
                              c3d_mc.outputs.itk_transform]

        norm = ApplyTransforms(
            input_image='temp_func.nii.gz',
            reference_image=reference,
            transforms=transform_list,
            dimension=3,
            float=True,
            input_image_type=3,
            interpolation='LanczosWindowedSinc',
            invert_transform_flags=[False] * len(transform_list),
            output_image='temp_out.nii.gz',
            num_threads=1).run()

        out_list.append(load_img(norm.outputs.output_image))
        print(mat_files[i])

    # Concatenate image and add TR value to header
    imgs = concat_imgs(out_list)
    imgs = new_img_like(reference, imgs.get_data(), copy_header=True)
    imgs.header.set_zooms(list(imgs.header.get_zooms()[:3]) + [TR])
    imgs.to_filename(out_file)
    
    return out_file

apply_warp = Node(Function(input_names=[
    'in_file', 'mat_folder', 'coreg', 'brain', 'transforms',
    'template', 'norm_func', 't_min', 'TR'],
                              output_names=['out_file'],
                              function=apply_warps),
                     name='apply_warp')
apply_warp.inputs.norm_func = norm_func

### Image coregistration subworkflow

In [ ]:
# Pre-alignment of functional images to anatomical image
coreg_pre = Node(FLIRT(dof=6,
                       output_type='NIFTI_GZ'),
                 name='coreg_pre')

In [ ]:
# Coregistration of functional images to anatomical image with BBR
# using WM segmentation
coreg_bbr = Node(FLIRT(dof=9,
                       cost='bbr',
                       schedule=opj(os.getenv('FSLDIR'),
                                    'etc/flirtsch/bbr.sch'),
                       output_type='NIFTI_GZ'),
                 name='coreg_bbr')

In [ ]:
# Create coregistration workflow
coregflow = Workflow(name='coregflow')

# Add nodes to workflow and connect them
coregflow.connect([(coreg_pre, coreg_bbr, [('out_matrix_file', 'in_matrix_file')])])

### Temporal and spatial filter subworkflow

In [ ]:
# Create again a brain mask for the functional image and one for the confounds
def create_warped_mask(in_file):

    import numpy as np
    from nipype.interfaces.fsl import BET
    from nipype.interfaces.ants import N4BiasFieldCorrection
    from nilearn.image import mean_img, new_img_like, load_img
    from scipy.ndimage import binary_dilation, binary_erosion, binary_fill_holes
    from os.path import basename, abspath

    # Compute mean image
    mean_file = abspath(basename(in_file).replace('.nii', '_mean.nii'))
    mean_img(in_file).to_filename(mean_file)

    # Apply N4BiasFieldCorrection on mean file
    res = N4BiasFieldCorrection(input_image=mean_file,
                                dimension=3, copy_header=True).run()

    # Create brain mask based on functional bias corrected mean file
    res = BET(in_file=res.outputs.output_image, mask=True,
              no_output=True, robust=True).run()

    # Dilate the brain mask twice and fill wholes for functional mask
    brain = load_img(res.outputs.mask_file).get_data()
    mask_func = binary_fill_holes(binary_dilation(brain, iterations=2))

    # Dilate brain mask once, fill wholes and erode twice for confound mask
    mask_conf = binary_erosion(binary_fill_holes(
            binary_dilation(brain, iterations=1)), iterations=2)

    # Warping an image can induce noisy new voxels in the edge regions 
    # of a slab, which can be problematic for temporal filtering or
    # later ICA. For this reason, we first drop any voxels that have
    # zero-activation in more than 1% of all volumes and combine this
    # with our previous brain mask
    def remove_zero_voxels(in_file, bin_thr=1, vol_thr=0.99):
        data = np.abs(load_img(in_file).get_data())
        bins = np.histogram_bin_edges(np.ravel(data[data>0]), bins=100)
        bin_cutoff = bins[bin_thr]
        mask_zeros = np.sum(data>bin_cutoff, axis=-1)>(data.shape[-1] * vol_thr)
        return binary_fill_holes(mask_zeros)

    # Combine the functional brain mask with zero voxel mask and fill holes
    mask_zeros = remove_zero_voxels(in_file, bin_thr=1, vol_thr=0.99)
    data_mask = mask_zeros * mask_func
    mask_func = binary_fill_holes(data_mask)

    # Combine the confound brain mask with zero voxel mask, dilate once,
    # fill wholes and erode twice
    mask_zeros = remove_zero_voxels(in_file, bin_thr=5, vol_thr=0.95)
    data_mask = mask_zeros * mask_conf
    mask_conf = binary_erosion(binary_fill_holes(
        binary_dilation(data_mask, iterations=1)), iterations=2)

    # Save masks as NIfTI images
    img_mask_func = new_img_like(in_file, mask_func.astype('int'),
                                 copy_header=True)
    mask_func = abspath(basename(in_file).replace('.nii', '_mask_func.nii'))
    img_mask_func.to_filename(mask_func)

    img_mask_conf = new_img_like(in_file, mask_conf.astype('int'),
                                 copy_header=True)
    mask_conf = abspath(basename(in_file).replace('.nii', '_mask_conf.nii'))
    img_mask_conf.to_filename(mask_conf)

    return mask_func, mask_conf

masks_for_warp = Node(Function(input_names=['in_file'],
                              output_names=['mask_func', 'mask_conf'],
                              function=create_warped_mask),
                     name='masks_for_warp')
masks_for_warp.inputs.norm_func = norm_func

In [ ]:
# Apply temporal filter to functional image
def apply_temporal_filter(in_file, mask, tFilter, tr):

    import numpy as np
    from nipype.interfaces.afni import Bandpass
    from nilearn.image import load_img, math_img, mean_img, new_img_like
    from os.path import basename, abspath

    # Extract low- and high-pass filter
    low_pass = tFilter[0]
    high_pass = tFilter[1]
    lowpass = 1. / low_pass if low_pass != None else 999999
    highpass = 1. / high_pass if high_pass != None else 0

    # Temporal filtering to get rid of high and/or low-pass frequencies
    res = Bandpass(in_file=in_file,
                   mask=mask,
                   lowpass=lowpass,
                   highpass=highpass,
                   tr=tr,
                   num_threads=-1,
                   no_detrend=True,
                   outputtype='NIFTI_GZ').run()
    
    # Add mean image back to functional image and apply mask
    img_mean = mean_img(in_file)
    img_out = math_img(
        '(img + mean[..., None]) * mask[..., None]', mask=mask,
        img=res.outputs.out_file, mean=img_mean)

    # Intensity normalize image to the white matter histogram density peak
    img_mean = mean_img(img_out)
    count, bins = np.histogram(np.ravel(np.abs(img_mean.get_data())), bins=128)
    sigma = bins[32 + np.argmax(count[32:])]
    sigma /= 10000
    data = img_out.get_data() / sigma

    # Save output into NIfTI file
    img_out = new_img_like(in_file, data, copy_header=True)
    out_file = abspath(basename(in_file).replace('.nii', '_tf.nii'))
    img_out.to_filename(out_file)

    mean_file = abspath(basename(in_file).replace('.nii', '_tf_mean.nii'))
    img_mean.to_filename(mean_file)
    
    return out_file, mean_file

temporal_filter = Node(Function(input_names=['in_file', 'mask', 'tFilter', 'tr'],
                                 output_names=['out_file', 'mean_file'],
                                 function=apply_temporal_filter),
                        name='temporal_filter')

In [ ]:
# Applies gaussian spatial filter as in Sengupta, Pollmann & Hanke, 2018
def gaussian_spatial_filter(in_file, sFilter, mask, bandwidth=2):

    import numpy as np
    from nilearn.image import load_img, smooth_img, math_img, new_img_like
    from os.path import basename, abspath

    # Extract smoothing type and FWHM value
    ftype, fwhm = sFilter
    
    if fwhm == 0:
        img = load_img(in_file)

    elif ftype == 'LP':
        img = smooth_img(in_file, fwhm=fwhm)
        
    elif ftype == 'HP':
        img_smooth = smooth_img(in_file, fwhm=fwhm)
        img = math_img('img1 - img2', img1=img_smooth, img2=in_file)
        
    elif ftype == 'BP':
        img_smooth_high = smooth_img(in_file, fwhm=fwhm)
        img_smooth_low = smooth_img(in_file, fwhm=fwhm - bandwidth)
        img = math_img('img1 - img2', img1=img_smooth_high, img2=img_smooth_low)

    # Mask smoothed image
    mask = load_img(mask).get_data()
    data = img.get_data() * mask[..., None]
        
    # Before we can save the final output NIfTI in 'int16' format, we need
    # to make sure that there's no data overflow, i.e. values above 32768
    data = img.get_data()
    max_value = 30000
    max_data = np.max(np.abs(data))
    if max_data > max_value:
        data /= max_data
        data *= max_value
        print('Max-value was adapted: From %f to %f' % (max_data, max_value))
    
    # Now we can reset the header and save image to file with data type 'int'
    out_img = new_img_like(in_file, data.astype('int16'), copy_header=True)
    out_img.set_data_dtype('int16')   
    out_file = abspath(basename(in_file).replace('.nii', '_%s_%smm.nii' % (ftype, fwhm)))
    out_img.to_filename(out_file)

    return out_file

# Spatial Band-Pass Filter
spatial_filter = Node(Function(input_names=['in_file', 'sFilter', 'mask'],
                               output_names=['out_file'],
                               function=gaussian_spatial_filter),
                      name='spatial_filter')
spatial_filter.iterables = ('sFilter', filters_spatial)

In [ ]:
# Create temporal and spatial filter workflow
filterflow = Workflow(name='filterflow')

# Add nodes to workflow and connect them
filterflow.connect([(masks_for_warp, temporal_filter, [('mask_func', 'mask')]),
                    (masks_for_warp, spatial_filter, [('mask_func', 'mask')]),
                    (temporal_filter, spatial_filter, [('out_file', 'in_file')]),
                   ])

### Create Main Workflow

**Note:** Slice time correction is applied after motion correction, as recommended by Power et al. (2017): http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0182939

In [ ]:
# Create main preprocessing workflow
mainflow = Workflow(name='mainflow')

# Add nodes to workflow and connect them
mainflow.connect([(prepareflow, estimate_motion, [('reorient.out_file', 'in_file'),
                                                  ('mask_func_brain.mask_file', 'mask_file'),
                                                  ('nss_detection.t_min', 't_min'),
                                                 ]),
                  (estimate_motion, motion_parameters, [('mean_file', 'mean_file'),
                                                        ('par_file', 'par_file'),
                                                        ('mat_folder', 'mat_folder')]),
                  (prepareflow, slice_time, [('reorient.out_file', 'in_files')]),
                  (slice_time, apply_warp, [('timecorrected_files', 'in_file')]),
                  (prepareflow, apply_warp, [('nss_detection.t_min', 't_min')]),
                  (estimate_motion, coregflow, [('mean_file', 'coreg_pre.in_file'),
                                                ('mean_file', 'coreg_bbr.in_file')]),
                  (coregflow, apply_warp, [('coreg_bbr.out_matrix_file', 'coreg')]),
                  (motion_parameters, apply_warp, [('mat_folder', 'mat_folder')]),
                  (apply_warp, filterflow, [('out_file', 'masks_for_warp.in_file'),
                                            ('out_file', 'temporal_filter.in_file')]),
                  ])

## Create a subworkflow for the Confound Workflow

### Implement Nodes

In [ ]:
# Run ACompCor (based on Behzadi et al., 2007)
aCompCor = Node(ACompCor(num_components=n_compcor_confounds,
                         pre_filter='cosine',
                         save_pre_filter=False,
                         merge_method='union',
                         components_file='compcorA.txt'),
                name='aCompCor')

In [ ]:
# Create binary mask for ACompCor (based on Behzadi et al., 2007)
def get_csf_wm_mask(mean_file, wm, csf, brainmask, 
                    temp_wm, temp_csf, norm_func):

    from os.path import basename, abspath
    from nilearn.image import load_img, threshold_img, resample_to_img, new_img_like
    from scipy.ndimage.morphology import binary_erosion, binary_closing

    # Specify name of output file
    out_file = abspath(basename(mean_file).replace('.nii', '_maskA.nii'))
    
    if norm_func:

        # Create eroded WM binary mask
        bin_wm = threshold_img(temp_wm, 0.5)
        mask_wm = binary_erosion(bin_wm.get_data(), iterations=2).astype('int8')

        # Create eroded CSF binary mask (differs from Behzadi et al., 2007)
        bin_csf = threshold_img(temp_csf, 0.5)
        close_csf = binary_closing(bin_csf.get_data(), iterations=1)
        mask_csf = binary_erosion(close_csf, iterations=1).astype('int8')
        
    else:
    
        # Create eroded WM binary mask
        thr_wm = resample_to_img(threshold_img(wm, 0.99), mean_file)
        bin_wm = threshold_img(thr_wm, 0.5)
        mask_wm = binary_erosion(bin_wm.get_data(), iterations=2).astype('int8')

        # Create eroded CSF binary mask (differs from Behzadi et al., 2007)
        thr_csf = resample_to_img(threshold_img(csf, 0.99), mean_file)
        bin_csf = threshold_img(thr_csf, 0.5)
        close_csf = binary_closing(bin_csf.get_data(), iterations=1)
        mask_csf = binary_erosion(close_csf, iterations=1).astype('int8')

    # Load brain mask
    mask_brain = load_img(brainmask).get_data()

    # Combine WM and CSF binary masks into one and apply brainmask
    binary_mask = (((mask_wm + mask_csf) * mask_brain) > 0).astype('int8')
    mask_img = new_img_like(mean_file, binary_mask.astype('int16'), copy_header=True)
    mask_img.to_filename(out_file)
    
    return out_file

acomp_masks = Node(Function(input_names=['mean_file', 'wm', 'csf', 'brainmask',
                                         'temp_wm', 'temp_csf', 'norm_func'],
                            output_names=['out_file'],
                            function=get_csf_wm_mask),
                   name='acomp_masks')
acomp_masks.inputs.norm_func = norm_func

In [ ]:
# Run TCompCor (based on Behzadi et al., 2007)
tCompCor = Node(TCompCor(num_components=n_compcor_confounds,
                         percentile_threshold=0.02,
                         pre_filter='cosine',
                         save_pre_filter=False,
                         components_file='compcorT.txt'),
                name='tCompCor')

In [ ]:
# Compute ICA components
def extract_ica_components(in_file, mask_file, n_components):
    
    import numpy as np
    from nilearn.image import load_img
    from scipy.stats import zscore, pearsonr
    from nilearn.decomposition import CanICA
    from os.path import basename, abspath

    # Load functiona image and mask
    img = load_img(in_file)
    img_mask= load_img(mask_file)

    # Compute average inplane resolution for light smoothing
    fwhm = np.mean(img.header.get_zooms()[:2])

    # Specify CanICA object
    canica = CanICA(n_components=n_components, smoothing_fwhm=fwhm,
                    mask=mask_file, threshold='auto', n_jobs=1,
                    standardize=True, detrend=True)

    # Fit CanICA on input data
    canica.fit(img)
    
    # Save components into NIfTI file
    comp_file = abspath(basename(in_file).replace('.nii', '_ICA_comp.nii'))
    img_comp = canica.components_img_
    img_comp.to_filename(comp_file)

    # Extract data and mask from images
    data = img.get_data()
    mask = img_mask.get_data()!=0

    # Compute the pearson correlation between the components and the signal
    curves = zscore([[pearsonr(img_comp.get_data()[mask, j],
                               data[mask, i])[0] for i in range(data.shape[-1])]
                     for j in range(n_components)], axis=-1)
    comp_signal = abspath(basename(in_file).replace('.nii.gz', '_ICA_comp.txt'))
    np.savetxt(comp_signal, curves, fmt='%.8e', delimiter=' ', newline='\n')

    return comp_file, comp_signal

compute_ica = Node(Function(input_names=['in_file', 'mask_file', 'n_components'],
                            output_names=['comp_file', 'comp_signal'],
                            function=extract_ica_components),
                   name='compute_ica')
compute_ica.inputs.n_components = n_independent_components

In [ ]:
# Compute framewise displacement (FD)
FD = Node(FramewiseDisplacement(parameter_source='FSL',
                                normalize=False),
          name='FD')

In [ ]:
# Compute DVARS
dvars = Node(ComputeDVARS(remove_zerovariance=True,
                          save_vxstd=True),
             name='dvars')

In [ ]:
# Computes Friston 24-parameter model (Friston et al., 1996)
def compute_friston24(in_file):
    
    import numpy as np
    from os.path import basename, abspath
    
    # Load raw motion parameters
    mp_raw = np.loadtxt(in_file)
    
    # Get motion paremter one time point before (first order difference)
    mp_minus1 = np.vstack(([0] * 6, mp_raw[1:]))
    
    # Combine the two
    mp_combine = np.hstack((mp_raw, mp_minus1))

    # Add the square of those parameters to allow correction of nonlinear effects
    mp_friston = np.hstack((mp_combine, mp_combine**2))

    # Save friston 24-parameter model in new txt file
    out_file = abspath(basename(in_file).replace('.txt', 'friston24.txt'))
    np.savetxt(out_file, mp_friston,
               fmt='%.8e', delimiter=' ', newline='\n')
    
    return out_file

friston24 = Node(Function(input_names=['in_file'],
                          output_names=['out_file'],
                          function=compute_friston24),
                 name='friston24')

In [ ]:
# Compute average signal in total volume, in GM, in WM and in CSF
def get_average_signal(in_file, gm, wm, csf, brainmask, template_file,
                       temp_mask, temp_gm, temp_wm, temp_csf, norm_func):

    from scipy.stats import zscore
    from nilearn.image import load_img, threshold_img, resample_to_img, math_img
    from nilearn.masking import apply_mask

    if norm_func:
        res_brain = temp_mask
        res_gm = threshold_img(temp_gm, 0.99)
        res_wm = threshold_img(temp_wm, 0.99)
        res_csf = threshold_img(temp_csf, 0.99)
        
    else:
        res_brain = resample_to_img(brainmask, template_file)
        res_gm = resample_to_img(threshold_img(gm, 0.99), template_file)
        res_wm = resample_to_img(threshold_img(wm, 0.99), template_file)
        res_csf = resample_to_img(threshold_img(csf, 0.99), template_file)

    
    # Create masks for signal extraction
    bin_brain = math_img('(mask>=0.5) * template',
                         mask=res_brain, template=template_file)
    bin_gm = math_img('(mask>=0.5) * template',
                      mask=res_gm, template=template_file)
    bin_wm = math_img('(mask>=0.5) * template',
                      mask=res_wm, template=template_file)
    bin_csf = math_img('(mask>=0.5) * template',
                       mask=res_csf, template=template_file)

    # Load data from functional image and zscore it
    img = load_img(in_file)

    # Compute average signal per mask and zscore timeserie
    signal_gm = zscore(apply_mask(img, bin_gm).mean(axis=1))
    signal_wm = zscore(apply_mask(img, bin_wm).mean(axis=1))
    signal_csf = zscore(apply_mask(img, bin_csf).mean(axis=1))
    signal_brain = zscore(apply_mask(img, bin_brain).mean(axis=1))

    return [signal_brain, signal_gm, signal_wm, signal_csf]

average_signal = Node(Function(input_names=[
    'in_file', 'gm', 'wm', 'csf', 'brainmask', 'template_file',
    'temp_mask', 'temp_gm', 'temp_wm', 'temp_csf', 'norm_func'],
                               output_names=['average'],
                               function=get_average_signal),
                      name='average_signal')
average_signal.inputs.norm_func = norm_func

In [ ]:
# Combine confound parameters into one TSV file
def consolidate_confounds(FD, DVARS, par_mc, par_mc_raw, par_friston,
                          compA, compT, average, ica_comp):
    
    import numpy as np
    from os.path import basename, abspath
    
    conf_FD = np.array([0] + list(np.loadtxt(FD, skiprows=1)))
    conf_DVARS = np.array([1] + list(np.loadtxt(DVARS, skiprows=0)))
    conf_mc = np.loadtxt(par_mc)
    conf_mc_raw = np.loadtxt(par_mc_raw)
    conf_friston = np.loadtxt(par_friston)
    conf_compA = np.loadtxt(compA, skiprows=1)
    conf_compT = np.loadtxt(compT, skiprows=1)
    conf_average = np.array(average)
    conf_ica = np.loadtxt(ica_comp).T

    # Aggregate confounds
    confounds = np.hstack((conf_FD[..., None],
                           conf_DVARS[..., None],
                           conf_average.T,
                           conf_mc,
                           conf_mc_raw,
                           conf_friston,
                           conf_ica,
                           conf_compA,
                           conf_compT))

    # Create header
    header = ['FD', 'DVARS']
    header += ['TV', 'GM', 'WM', 'CSF']
    header += ['Rotation%02d' % (d + 1) for d in range(3)]
    header += ['Translation%02d' % (d + 1) for d in range(3)]
    header += ['Rotation%02d_raw' % (d + 1) for d in range(3)]
    header += ['Translation%02d_raw' % (d + 1) for d in range(3)]
    header += ['Friston%02d' % (d + 1) for d in range(conf_friston.shape[1])]
    header += ['ICA%02d' % (d + 1) for d in range(conf_ica.shape[1])]
    header += ['CompA%02d' % (d + 1) for d in range(conf_compA.shape[1])]
    header += ['CompT%02d' % (d + 1) for d in range(conf_compT.shape[1])]

    # Write to file
    out_file = abspath(basename(par_mc).replace('.par', '_confounds.tsv'))
    with open(out_file, 'w') as f:
        f.write('\t'.join(header) + '\n')
        for row in confounds:
            f.write('\t'.join([str(r) for r in row]) + '\n')
    
    return out_file

combine_confounds = Node(Function(input_names=['FD', 'DVARS', 'par_mc', 'par_mc_raw',
                                               'par_friston', 'compA', 'compT',
                                               'average', 'ica_comp'],
                                  output_names=['out_file'],
                                  function=consolidate_confounds),
                         name='combine_confounds')

### Create Confound Workflow

In [ ]:
# Create confound extraction workflow
confflow = Workflow(name='confflow')

# Add nodes to workflow and connect them
confflow.connect([(acomp_masks, aCompCor, [('out_file', 'mask_files')]),

                  # Consolidate confounds
                  (FD, combine_confounds, [('out_file', 'FD')]),
                  (dvars, combine_confounds, [('out_vxstd', 'DVARS')]),
                  (aCompCor, combine_confounds, [('components_file', 'compA')]),
                  (tCompCor, combine_confounds, [('components_file', 'compT')]),
                  (friston24, combine_confounds, [('out_file', 'par_friston')]),
                  (average_signal, combine_confounds, [('average', 'average')]),
                  (compute_ica, combine_confounds, [('comp_signal', 'ica_comp')]),
                  ])

## Create a subworkflow for the report Workflow

### Implement Nodes

In [ ]:
# Plot mean image with brainmask and ACompCor and TCompCor mask ovleray
def plot_masks(sub_id, ses_id, task_id, run_id, mean, maskA, maskT, brainmask):
    
    import numpy as np
    import nibabel as nb
    from matplotlib.pyplot import figure
    from nilearn.plotting import plot_roi, find_cut_slices
    from os.path import basename, abspath

    # If needed, create title for output figures
    title_txt = 'Sub: %s - Task: %s' % (sub_id, task_id)
    if ses_id:
        title_txt += ' - Sess: %s' % ses_id
    if run_id:
        title_txt += ' - Run: %d' % run_id

    # Establish name of output file
    out_file = basename(mean).replace('_mean.nii.gz', '_overlays.png')

    # Prepare maskA, maskT and brainmask (otherwise they create strange looking outputs)
    img = nb.load(mean)
    data = np.stack((np.zeros(img.shape),
                    nb.load(brainmask).get_data(),
                    nb.load(maskA).get_data() * 2,
                    nb.load(maskT).get_data() * 3),
                    axis= -1)
    label_id = np.argmax(data, axis=-1)
    masks = nb.Nifti1Image(label_id, img.affine, img.header)

    # Get content extent of mean img and crop all images with it
    content = np.nonzero(img.get_data())
    c = np.ravel([z for z in zip(np.min(content, axis=1), np.max(content, axis=1))])
    img = img.slicer[c[0]:c[1], c[2]:c[3], c[4]:c[5]]
    masks = masks.slicer[c[0]:c[1], c[2]:c[3], c[4]:c[5]]

    # Plot functional mean and different masks used (compcor and brainmask)
    fig = figure(figsize=(16, 8))

    from matplotlib.colors import ListedColormap
    colormap = ListedColormap([(0.86, 0.3712, 0.34),
                               (0.3712, 0.34, 0.86),
                               (0.34, 0.86, 0.3712)])

    for i, e in enumerate(['x', 'y', 'z']):
        ax = fig.add_subplot(3, 1, i + 1)
        cuts = find_cut_slices(img, direction=e, n_cuts=10)[1:-1]
        plot_roi(masks, cmap=colormap, dim=1, annotate=False, bg_img=img,
                 display_mode=e, title=title_txt + ' - %s-axis' % e,
                 resampling_interpolation='nearest', cut_coords=cuts,
                 axes=ax, alpha=0.66)

    # Establish name of output file
    out_file = abspath(basename(mean).replace('_mean.nii.gz', '_overlays.png'))
    fig.savefig(out_file, bbox_inches='tight', facecolor='black',
                frameon=True, dpi=300, transparent=False)

    return out_file

compcor_plot = Node(Function(input_names=['sub_id', 'ses_id', 'task_id', 'run_id',
                                          'mean', 'maskA', 'maskT', 'brainmask'],
                             output_names=['out_file'],
                             function=plot_masks),
                    name='compcor_plot')

In [ ]:
# Plot confounds and detect outliers
def plot_confounds(confounds, outlier_thr):

    # This plotting is heavily based on MRIQC's visual reports (credit to oesteban)
    import numpy as np
    import pandas as pd
    from scipy.stats import zscore
    from matplotlib.backends.backend_pdf import FigureCanvasPdf as FigureCanvas
    import seaborn as sns
    sns.set(style="darkgrid")
    from matplotlib import pyplot as plt
    from matplotlib.gridspec import GridSpec
    from os.path import basename, abspath

    def plot_timeseries(dataframe, elements, out_file, outlier_thr=None, motion=False):

        # Number of rows to plot
        n_rows = len(elements)

        # Prepare for motion plot
        if motion:
            n_rows = int(n_rows / 2)

        # Create canvas
        fig = plt.Figure(figsize=(16, 2 * n_rows))
        FigureCanvas(fig)
        grid = GridSpec(n_rows, 2, width_ratios=[7, 1])

        # Specify color palette to use
        colors = sns.husl_palette(n_rows)

        # To collect possible outlier indices
        outlier_idx = []

        # Plot timeseries (and detect outliers, if specified)
        for i, e in enumerate(elements[:n_rows]):

            # Extract timeserie values
            data = dataframe[e].values

            # Z-score data for later thresholding
            zdata = zscore(data)

            # Plot timeserie
            ax = fig.add_subplot(grid[i, :-1])
            if motion:
                ax.plot(dataframe[e + '_raw'].values, color=[0.66] * 3)
            ax.plot(data, color=colors[i])
            ax.set_xlim((0, len(data)))
            ax.set_ylabel(e)
            ylim = ax.get_ylim()

            # Detect and plot outliers if threshold is specified
            if outlier_thr:

                threshold = outlier_thr[i]

                if threshold != None:

                    outlier_id = np.where(np.abs(zdata)>=threshold)[0]
                    outlier_idx += list(outlier_id)
                    ax.vlines(outlier_id, ylim[0], ylim[1])

            # Plot observation distribution
            ax = fig.add_subplot(grid[i, -1])
            sns.distplot(data, vertical=True, ax=ax, color=colors[i])
            ax.set_ylim(ylim)

        fig.tight_layout()
        fig.savefig(out_file)

        return np.unique(outlier_idx)

    # Load confounds table
    df = pd.read_csv(confounds, sep='	')
    df.fillna(0, inplace=True)

    # Aggregate output plots
    out_plots = []
    confounds = basename(confounds)
    
    # Plot main confounds
    elements = ['FD', 'DVARS', 'TV', 'GM', 'WM', 'CSF']
    out_file = abspath(confounds.replace('.tsv', '_main.png'))
    out_plots.append(out_file)
    outliers = plot_timeseries(df, elements, out_file, outlier_thr)
    
    # Save outlier indices to textfile
    outlier_filename = abspath(confounds.replace('.tsv', '_outliers.txt'))
    np.savetxt(outlier_filename, outliers, fmt='%d')

    # Plot Motion Paramters
    elements = [k for k in df.keys() if 'Rotation' in k or 'Translation' in k]
    out_file = abspath(confounds.replace('.tsv', '_motion.png'))
    out_plots.append(out_file)
    plot_timeseries(df, elements, out_file, motion=True)

    # Plot CompCor components
    for comp in ['A', 'T']:
        elements = [k for k in df.keys() if 'Comp%s' % comp in k]
        out_file = abspath(confounds.replace('.tsv', '_comp%s.png' % comp))
        out_plots.append(out_file)
        plot_timeseries(df, elements, out_file)
    
    # Reset seaborn
    sns.reset_orig()

    return [outlier_filename] + out_plots

confound_inspection = Node(Function(input_names=['confounds', 'outlier_thr'],
                                    output_names=['outlier_file', 'plot_main', 'plot_motion',
                                                  'plot_compA', 'plot_compT'],
                                    function=plot_confounds),
                           name='confound_inspection')
confound_inspection.inputs.outlier_thr = outlier_thr

In [ ]:
# Creates carpet plot
def create_carpet_plot(in_file, sub_id, ses_id, task_id, run_id, 
                       seg_gm, seg_wm, seg_csf, nVoxels, brainmask):

    from os.path import basename, abspath
    from nilearn.image import load_img, resample_to_img
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.stats import zscore
    import seaborn as sns

    # Load functional image and mask
    img = load_img(in_file)
    data = img.get_data()
    mask = load_img(brainmask).get_data()

    # Resample masks to functional space and threshold them
    mask_gm = resample_to_img(seg_gm, img, interpolation='nearest').get_data() >= 0.5
    mask_wm = resample_to_img(seg_wm, img, interpolation='nearest').get_data() >= 0.5
    mask_csf = resample_to_img(seg_csf, img, interpolation='nearest').get_data() >= 0.5

    # Restrict signal to plot to specific mask
    data_gm = data[(mask_gm * mask).astype('bool')]
    data_wm = data[(mask_wm * mask).astype('bool')]
    data_csf = data[(mask_csf * mask).astype('bool')]

    # Remove voxels without any variation over time
    data_gm = data_gm[data_gm.std(axis=-1)!=0]
    data_wm = data_wm[data_wm.std(axis=-1)!=0]
    data_csf = data_csf[data_csf.std(axis=-1)!=0]

    # Compute stepsize and reduce datasets
    stepsize = int((len(data_gm) + len(data_wm) + len(data_csf)) / nVoxels)
    data_gm = data_gm[::stepsize]
    data_wm = data_wm[::stepsize]
    data_csf = data_csf[::stepsize]

    # Sort voxels according to correlation to mean signal within a ROI
    data_gm = data_gm[np.argsort([np.corrcoef(d, data_gm.mean(axis=0))[0, 1] for d in data_gm])]
    data_wm = data_wm[np.argsort([np.corrcoef(d, data_wm.mean(axis=0))[0, 1] for d in data_wm])]
    data_csf = data_csf[np.argsort([np.corrcoef(d, data_csf.mean(axis=0))[0, 1] for d in data_csf])]

    # Create carpet plot, zscore and rescale it
    carpet = np.row_stack((data_gm, data_wm, data_csf))
    carpet = np.nan_to_num(zscore(carpet, axis=-1))
    carpet /= np.abs(carpet).max(axis=0)

    # Create title for figure
    title_txt = 'Sub: %s - Task: %s' % (sub_id, task_id)
    if ses_id:
        title_txt += ' - Sess: %s' % ses_id
    if run_id:
        title_txt += ' - Run: %d' % run_id
    
    # Plot carpet plot and save it
    fig = plt.figure(figsize=(12, 6))
    plt.imshow(carpet, aspect='auto', cmap='gray')
    plt.hlines((data_gm.shape[0]), 0, carpet.shape[1] - 1, colors='r')
    plt.hlines((data_gm.shape[0] + data_wm.shape[0]), 0, carpet.shape[1] - 1, colors='b')
    plt.title(title_txt)
    plt.xlabel('Volume')
    plt.ylabel('Voxel')
    plt.tight_layout()
    out_file = abspath(basename(in_file).replace('.nii.gz', '_carpet.png'))
    fig.savefig(out_file)

    # Reset seaborn
    sns.reset_orig()

    return out_file

carpet_plot = Node(Function(input_names=['in_file', 'sub_id', 'ses_id', 'task_id', 'run_id',
                                         'seg_gm', 'seg_wm', 'seg_csf', 'nVoxels', 'brainmask'],
                            output_names=['out_file'],
                            function=create_carpet_plot),
                   name='carpet_plot')
carpet_plot.inputs.nVoxels = 6000

In [ ]:
# Creates carpet plot
def plot_ica_components(comp_signal, comp_file, mean_file, TR):

    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_pdf import FigureCanvasPdf as FigureCanvas
    from matplotlib.gridspec import GridSpec
    import seaborn as sns
    sns.set(style="darkgrid")
    import numpy as np
    from nilearn.image import iter_img, load_img, coord_transform
    from nilearn.plotting import plot_stat_map, find_cut_slices
    from scipy.signal import welch
    from os.path import basename, abspath

    # Read data
    img_comp = load_img(comp_file)
    comp_data = np.loadtxt(comp_signal)
    n_components = comp_data.shape[0]
    elements = ['ICA%02d' % (d + 1) for d in range(n_components)]

    # Plot singal components and their power spectrum density maps
    fig = plt.Figure(figsize=(16, 2 * n_components))
    FigureCanvas(fig)
    grid = GridSpec(n_components, 2, width_ratios=[6, 2])

    # Specify color palette to use
    colors = sns.husl_palette(n_components)

    # Plot timeseries
    freq, power_spectrum = welch(comp_data, fs=1. / TR)
    for i, e in enumerate(elements):

        # Extract timeserie values
        data = comp_data[i].T

        # Plot timeserie
        ax = fig.add_subplot(grid[i, :-1])
        ax.plot(data, color=colors[i])
        ax.set_xlim((0, len(data)))
        ax.set_ylabel(e)
        ylim = ax.get_ylim()

        # Plot power density spectrum of all components
        ax = fig.add_subplot(grid[i, -1])
        ax.plot(freq, power_spectrum[i], color=colors[i])

    fig.tight_layout()

    # Save everyting in output figure
    fig_signal = abspath(basename(comp_signal).replace('.txt', '_signal.png'))
    fig.savefig(fig_signal, bbox_inches='tight', frameon=True, dpi=300, transparent=False)

    # Plot individual components on functional mean image
    fig = plt.figure(figsize=(16, 2 * n_components))
    for i, cur_img in enumerate(iter_img(img_comp)):
        ax = fig.add_subplot(n_components, 1, i + 1)
        cuts = find_cut_slices(cur_img, direction='z', n_cuts=12)[1:-1]
        plot_stat_map(cur_img, title='%s' % elements[i], colorbar=False,
                      threshold=np.abs(cur_img.get_data()).max() * 0.1,
                      bg_img=mean_file, display_mode='z', dim=0,
                      cut_coords=cuts, annotate=False, axes=ax)

    fig_brain = abspath(basename(comp_signal).replace('.txt', '_brain.png'))
    fig.savefig(fig_brain, bbox_inches='tight', facecolor='black', transparent=False)
    
    # Reset seaborn
    sns.reset_orig()
    
    return fig_signal, fig_brain

ica_plot = Node(Function(input_names=['comp_signal', 'comp_file', 'mean_file',
                                      'sub_id', 'ses_id', 'task_id', 'run_id', 'TR'],
                         output_names=['fig_signal', 'fig_brain'],
                         function=plot_ica_components),
                name='ica_plot')

In [ ]:
# Update report
def write_report(sub_id, ses_id, task_list, run_list, tFilter):

    # Load template for functional preprocessing output
    with open('/reports/report_template_preproc_func.html', 'r') as report:
        func_temp = report.read()

    # Create html filename for report
    html_file = '/data/derivatives/fmriflows/sub-%s.html' % sub_id
    if ses_id:
        html_file = html_file.replace('.html', '_ses-%s.html' % ses_id)

    # Old template placeholder
    func_key = '<p>The functional preprocessing pipeline hasn\'t been run yet.</p>'
    
    # Add new content to report
    with open(html_file, 'r') as report:
        txt = report.read()
        
        # Reset report with functional preprocessing template
        cut_start = txt.find('Functional Preprocessing</a></h2>') + 33
        cut_stop = txt.find('<!-- Section: 1st-Level Univariate Results-->')
        txt = txt[:cut_start] + func_key + txt[cut_stop:]

        txt_amendment = ''

        # Go through the placeholder variables and replace them with values
        for task_id in task_list:
            
            for t_filt in tFilter:
            
                if run_list:
                    for run_id in run_list:

                        func_txt = func_temp.replace('sub-placeholder', 'sub-%s' % sub_id)
                        func_txt = func_txt.replace('task-placeholder', 'task-%s' % task_id)
                        func_txt = func_txt.replace('run-placeholder', 'run-%02d' % run_id)
                        func_txt = func_txt.replace(
                            'tFilter_placeholder', 'tFilter_%s.%s' % (
                                str(t_filt[0]), str(t_filt[1])))
                        
                        if ses_id:
                            func_txt = func_txt.replace(
                                'ses-placeholder', 'ses-%s' % ses_id)
                        else:
                            func_txt = func_txt.replace('ses-placeholder', '')
                            func_txt = func_txt.replace('__', '_')

                        txt_amendment += func_txt

                else:

                    func_txt = func_temp.replace('sub-placeholder', 'sub-%s' % sub_id)
                    func_txt = func_txt.replace('task-placeholder', 'task-%s' % task_id)
                    func_txt = func_txt.replace('run-placeholder', '')
                    func_txt = func_txt.replace(
                            'tFilter_placeholder', 'tFilter_%s.%s' % (
                                str(t_filt[0]), str(t_filt[1])))

                    func_txt = func_txt.replace('__', '_')

                    if ses_id:
                        func_txt = func_txt.replace(
                            'ses-placeholder', 'ses-%s' % ses_id)
                    else:
                        func_txt = func_txt.replace('ses-placeholder', '')
                        func_txt = func_txt.replace('__', '_')

                    txt_amendment += func_txt
 
    # Add pipeline graphs
    txt_amendment += '<h3 class="h3" style="position:left;font-weight:bold">Graph of'
    txt_amendment += ' Functional Preprocessing pipeline</h3>\n    <object data="preproc_func/graph.png"'
    txt_amendment += ' type="image/png+xml" style="width:100%"></object>\n  '
    txt_amendment += ' <object data="preproc_func/graph_detailed.png" type="image/png+xml"'
    txt_amendment += ' style="width:100%"></object>\n'

    # Insert functional preprocessing report
    txt = txt.replace(func_key, txt_amendment)

    # Overwrite previous report
    with open(html_file, 'w') as report:
        report.writelines(txt)

create_report = Node(Function(input_names=['sub_id', 'ses_id', 'task_list',
                                           'run_list', 'tFilter'],
                              output_names=['out_file'],
                              function=write_report),
                     name='create_report')
create_report.inputs.run_list = run_list
create_report.inputs.task_list = task_list
create_report.inputs.tFilter = filters_temporal

### Create report Workflow

In [ ]:
# Create report workflow
reportflow = Workflow(name='reportflow')

# Add nodes to workflow and connect them
reportflow.add_nodes([compcor_plot,
                      confound_inspection,
                      create_report,
                      carpet_plot,
                      ica_plot])

## Specify Input & Output Stream

In [ ]:
# Iterate over subject, session, task and run id
info_source = Node(IdentityInterface(fields=['subject_id',
                                             'session_id',
                                             'task_id',
                                             'run_id']),
                   name='info_source')

iter_list = [('subject_id', subject_list),
             ('task_id', task_list)]

if session_list:
    iter_list.append(('session_id', session_list))
else:
    info_source.inputs.session_id = ''

if run_list:
    iter_list.append(('run_id', run_list))
else:
    info_source.inputs.run_id = ''

info_source.iterables = iter_list

In [ ]:
# Create path to input files
def create_file_path(subject_id, session_id, task_id, run_id):

    from bids.layout import BIDSLayout
    layout = BIDSLayout('/data/')

    # Find the right functional image
    search_parameters = {'subject': subject_id,
                         'return_type': 'file',
                         'suffix': 'bold',
                         'task': task_id,
                         'extensions': 'nii.gz',
                        }
    if session_id:
        search_parameters['session'] = session_id
    if run_id:
        search_parameters['run'] = run_id

    func = layout.get(**search_parameters)[0]

    # Collect structural images
    template_path = '/data/derivatives/fmriflows/preproc_anat/sub-{0}/sub-{0}_'
    if session_id:
        template_path += 'ses-%s_' % session_id
    template_anat = template_path + '{1}.nii.gz'
    
    # Collect normalization matrix
    trans_path = template_path + '{1}.h5'
    transforms = trans_path.format(subject_id, 'transformComposite')
    
    brain = template_anat.format(subject_id, 'brain')
    brainmask = template_anat.format(subject_id, 'brainmask')
    gm = template_anat.format(subject_id, 'seg_gm')
    wm = template_anat.format(subject_id, 'seg_wm')
    csf = template_anat.format(subject_id, 'seg_csf')
    
    return func, brain, brainmask, gm, wm, csf, transforms

select_files = Node(Function(input_names=['subject_id', 'session_id', 'task_id', 'run_id'],
                             output_names=['func', 'brain', 'brainmask', 'gm', 'wm', 'csf',
                                           'transforms'],
                             function=create_file_path),
                    name='select_files')

In [ ]:
# Compute Brain Mask and Extract Brain
def crop_images(brain, brainmask, gm, wm, csf):

    # Cropping image size to reduce memory load during coregistration
    from nilearn.image import crop_img, resample_img
    from os.path import basename, abspath
    
    brain_crop = crop_img(brain)
    affine = brain_crop.affine
    bshape = brain_crop.shape
    brainmask_crop = resample_img(brainmask, target_affine=affine, target_shape=bshape)
    gm_crop = resample_img(gm, target_affine=affine, target_shape=bshape)
    wm_crop = resample_img(wm, target_affine=affine, target_shape=bshape)
    csf_crop = resample_img(csf, target_affine=affine, target_shape=bshape)
    
    # Specify output name and save file
    brain_out = abspath(basename(brain))
    brainmask_out = abspath(basename(brainmask))
    gm_out = abspath(basename(gm))
    wm_out = abspath(basename(wm))
    csf_out = abspath(basename(csf))
    
    brain_crop.to_filename(brain_out)
    brainmask_crop.to_filename(brainmask_out)
    gm_crop.to_filename(gm_out)
    wm_crop.to_filename(wm_out)
    csf_crop.to_filename(csf_out)

    return brain_out, brainmask_out, gm_out, wm_out, csf_out

crop_brain = Node(Function(input_names=['brain', 'brainmask', 'gm', 'wm', 'csf'],
                           output_names=['brain', 'brainmask', 'gm', 'wm', 'csf'],
                           function=crop_images),
                  name='crop_brain')

In [ ]:
# Compute Brain Mask and Extract Brain
def create_templates(template_dir, res_norm):

    # Resample template brain to desired resolution
    from nibabel import load, Nifti1Image
    from nibabel.spaces import vox2out_vox
    from nilearn.image import resample_img
    from os.path import basename, abspath
    
    # Resample template images into requested resolution
    out_files = []
    for t in ['brain', 'mask', 'tpm_gm', 'tpm_wm', 'tpm_csf']:
        template = template_dir + '/1.0mm_%s.nii.gz' % t
        img = load(template)
        target_shape, target_affine = vox2out_vox(img, voxel_sizes=res_norm)
        img_resample = resample_img(img, target_affine, target_shape, clip=True)
        norm_template = abspath('template_{}_{}.nii.gz'.format(
            t, '_'.join([str(n) for n in res_norm])))
        img_resample.to_filename(norm_template)
        out_files.append(norm_template)

    return out_files

template_repository = Node(Function(input_names=['template_dir', 'res_norm'],
                                     output_names=['brain', 'mask',
                                                   'tpm_gm', 'tpm_wm', 'tpm_csf'],
                                     function=create_templates),
                  name='template_repository')

template_repository.inputs.template_dir = '/templates/mni_icbm152_nlin_asym_09c'
template_repository.inputs.res_norm = res_norm

In [ ]:
# Extract sequence specifications of functional images
def get_parameters(func, ref_slice):
    
    from bids.layout import BIDSLayout
    layout = BIDSLayout("/data/")
    parameter_info = layout.get_metadata(func)
    
    # Read out relevant parameters
    import numpy as np
    import nibabel as nb
    n_slices = nb.load(func).shape[2]
    TR = parameter_info['RepetitionTime']
    
    # If slice time onset are available, use them
    if 'SliceTiming' in parameter_info.keys():
        slice_order = parameter_info['SliceTiming']
        if np.mean(slice_order) <= 20:
            slice_order=[s*1000 for s in slice_order]
    else:
        # If not available, set time onset of all slices to zero
        slice_order = [0] * n_slices
    nslices = len(slice_order)
    time_acquisition = float(TR)-(TR/nslices)
    
    return TR, slice_order, nslices, time_acquisition

get_param = Node(Function(input_names=['func', 'ref_slice'],
                          output_names=['TR', 'slice_order',
                                        'nslices', 'time_acquisition'],
                          function=get_parameters),
                 name='get_param')
get_param.inputs.ref_slice = ref_timepoint

In [ ]:
# Iterate over the different temporal filters
def get_temporal_filters(tFilter):
    
    # Extract high-pass value for CompCor
    high_pass = tFilter[1] if tFilter[1] != None else 100.
    
    return tFilter, high_pass

get_tfilters = Node(Function(input_names=['tFilter'],
                          output_names=['tFilter', 'high_pass'],
                          function=get_temporal_filters),
                 name='get_tfilters')
get_tfilters.iterables = ('tFilter', filters_temporal)

In [ ]:
# Save relevant outputs in a datasink
datasink = Node(DataSink(base_directory=exp_dir,
                         container=out_dir),
                name='datasink')

In [ ]:
# Apply the following naming substitutions for the datasink
substitutions = [('/asub-', '/sub-'),
                 ('_bold', ''),
                 ('_ras', ''),
                 ('_tf', ''),
                 ('_mcf', ''),
                 ('_stc', ''),
                 ('_warped', ''),
                 ('.nii.gz_', '_'),
                 ('_mean_', '_'),
                 ('mask_000', 'maskT'),
                 ('.nii.gz.par', '.par'),
                ]

substitutions += [('tFilter_%s.%s/' % (t[0], t[1]),
                   'tFilter_%s.%s_' % (t[0], t[1]))
                  for t in filters_temporal]

substitutions += [('_sFilter_%s.%s/' % (s[0], s[1]), '')
                  for s in filters_spatial]

substitutions += [('%s_%smm' % (s[0], s[1]),
                   'sFilter_%s_%smm' % (s[0], s[1]))
                  for s in filters_spatial]                

for sub in subject_list:
    substitutions += [('sub-%s' % sub, '_')]

for sess in session_list:
    substitutions += [('ses-%s' % sess, '_')]

for task in task_list:
    substitutions += [('task-%s' % task, '_')]

for run in run_list:
    substitutions += [('run-%02d' % run, '_')]
    
for sub in subject_list:
    for task in task_list:

        substitutions += [('_subject_id_%s_task_id_%s/' % (sub, task),
                           'sub-{0}/sub-{0}_task-{1}_'.format(sub, task))]
        for sess in session_list:
            substitutions += [('_session_id_{0}sub-{1}/sub-{1}_task-{2}_'.format(sess, sub, task),
                               'sub-{0}/sub-{0}_ses-{1}_task-{2}_'.format(sub, sess, task))]
            for run in run_list:
                substitutions += [('_run_id_{0:d}sub-{1}/sub-{1}_ses-{2}_task-{3}_'.format(run, sub, sess, task),
                                   'sub-{0}/sub-{0}_ses-{1}_task-{2}_run-{3:02d}_'.format(sub, sess, task, run))]

        for run in run_list:
            substitutions += [('_run_id_{0:d}sub-{1}/sub-{1}_task-{2}_'.format(run, sub, task),
                               'sub-{0}/sub-{0}_task-{1}_run-{2:02d}_'.format(sub, task, run))]
            
substitutions += [('__', '_')] * 100
substitutions += [('_.', '.')]
datasink.inputs.substitutions = substitutions

## Create Functional Preprocessing Workflow

In [ ]:
# Create functional preprocessing workflow
preproc_func = Workflow(name='preproc_func')
preproc_func.base_dir = work_dir

# Connect input nodes to each other
preproc_func.connect([(info_source, select_files, [('subject_id', 'subject_id'),
                                                   ('session_id', 'session_id'),
                                                   ('task_id', 'task_id'),
                                                   ('run_id', 'run_id')]),
                      (select_files, crop_brain, [('brain', 'brain'),
                                                  ('brainmask', 'brainmask'),
                                                  ('gm', 'gm'),
                                                  ('wm', 'wm'),
                                                  ('csf', 'csf'),
                                                 ]),
                      (select_files, get_param, [('func', 'func')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the main workflow
preproc_func.connect([(crop_brain, mainflow, [('brain', 'coregflow.coreg_pre.reference'),
                                              ('brain', 'coregflow.coreg_bbr.reference'),
                                              ('wm', 'coregflow.coreg_bbr.wm_seg'),
                                             ]),
                      (get_param, mainflow, [('TR', 'slice_time.TR'),
                                             ('TR', 'filterflow.temporal_filter.tr'),
                                             ('TR', 'motion_parameters.TR'),
                                             ('TR', 'apply_warp.TR'),
                                             ('slice_order', 'slice_time.slice_order'),
                                             ('nslices', 'slice_time.nslices'),
                                             ('time_acquisition', 'slice_time.time_acquisition'),
                                            ]),
                      (get_tfilters, mainflow, [('tFilter', 'motion_parameters.tFilter'),
                                                ('tFilter', 'filterflow.temporal_filter.tFilter'),
                                               ]),
                      (select_files, mainflow, [('func', 'prepareflow.reorient.in_file'),
                                                ('transforms', 'apply_warp.transforms')]),
                      (template_repository, mainflow, [('brain', 'apply_warp.template')]),
                      (crop_brain, mainflow, [('brain', 'apply_warp.brain')]),
                      (mainflow, datasink, [
                          ('prepareflow.nss_detection.nss_file', 'preproc_func.@nss'),
                          ('estimate_motion.par_file', 'preproc_func.@par'),
                          ('motion_parameters.par_file', 'preproc_func.@par_filtered'),
                          ('filterflow.masks_for_warp.mask_func', 'preproc_func.@mask_func'),
                          ('filterflow.masks_for_warp.mask_conf', 'preproc_func.@mask_conf'),
                          ('filterflow.temporal_filter.mean_file', 'preproc_func.@mean'),
                          ('filterflow.spatial_filter.out_file', 'preproc_func.@func')]),
                     ])

In [ ]:
# Add input and output nodes and connect them to the confound workflow
preproc_func.connect([(crop_brain, confflow, [('brainmask', 'average_signal.brainmask'),
                                              ('gm', 'average_signal.gm'),
                                              ('wm', 'average_signal.wm'),
                                              ('csf', 'average_signal.csf'),
                                              ('wm', 'acomp_masks.wm'),
                                              ('csf', 'acomp_masks.csf')]),
                      (template_repository, confflow, [('mask', 'average_signal.temp_mask'),
                                                       ('tpm_gm', 'average_signal.temp_gm'),
                                                       ('tpm_wm', 'average_signal.temp_wm'),
                                                       ('tpm_csf', 'average_signal.temp_csf'),
                                                       ('tpm_wm', 'acomp_masks.temp_wm'),
                                                       ('tpm_csf', 'acomp_masks.temp_csf')]),
                      (get_param, confflow, [('TR', 'aCompCor.repetition_time'),
                                             ('TR', 'tCompCor.repetition_time'),
                                             ('TR', 'FD.series_tr'),
                                             ('TR', 'dvars.series_tr'),
                                            ]),
                      (get_tfilters, confflow, [('high_pass', 'aCompCor.high_pass_cutoff'),
                                                ('high_pass', 'tCompCor.high_pass_cutoff'),
                                               ]),
                      (confflow, datasink, [
                          ('tCompCor.high_variance_masks', 'preproc_func.@maskT'),
                          ('acomp_masks.out_file', 'preproc_func.@maskA'),
                          ('combine_confounds.out_file', 'preproc_func.@confound_tsv')
                      ]),
                     ])

In [ ]:
# Connect main workflow with confound workflow
preproc_func.connect([(mainflow, confflow, [
                          ('filterflow.temporal_filter.mean_file', 'acomp_masks.mean_file'),
                          ('filterflow.masks_for_warp.mask_conf', 'dvars.in_mask'),
                          ('filterflow.masks_for_warp.mask_conf', 'acomp_masks.brainmask'),
                          ('filterflow.masks_for_warp.mask_conf', 'tCompCor.mask_files'),
                          ('filterflow.masks_for_warp.mask_conf', 'average_signal.template_file'),
                          ('filterflow.masks_for_warp.mask_conf', 'compute_ica.mask_file'),
                          ('filterflow.temporal_filter.out_file', 'compute_ica.in_file'),
                          ('filterflow.temporal_filter.out_file', 'aCompCor.realigned_file'),
                          ('filterflow.temporal_filter.out_file', 'tCompCor.realigned_file'),
                          ('filterflow.temporal_filter.out_file', 'average_signal.in_file'),
                          ('filterflow.temporal_filter.out_file', 'dvars.in_file'),
                          ('motion_parameters.par_file', 'combine_confounds.par_mc'),
                          ('estimate_motion.par_file', 'combine_confounds.par_mc_raw'),
                          ('motion_parameters.par_file', 'friston24.in_file'),
                          ('motion_parameters.par_file', 'FD.in_file'),
                          ])
                     ])

In [ ]:
# Add input and output nodes and connect them to the report workflow
preproc_func.connect([(info_source, reportflow, [('subject_id', 'compcor_plot.sub_id'),
                                                 ('session_id', 'compcor_plot.ses_id'),
                                                 ('task_id', 'compcor_plot.task_id'),
                                                 ('run_id', 'compcor_plot.run_id'),

                                                 ('subject_id', 'create_report.sub_id'),
                                                 ('session_id', 'create_report.ses_id'),

                                                 ('subject_id', 'carpet_plot.sub_id'),
                                                 ('session_id', 'carpet_plot.ses_id'),
                                                 ('task_id', 'carpet_plot.task_id'),
                                                 ('run_id', 'carpet_plot.run_id'),
                                                ]),
                      (crop_brain, reportflow, [('gm', 'carpet_plot.seg_gm'),
                                                ('wm', 'carpet_plot.seg_wm'),
                                                ('csf', 'carpet_plot.seg_csf'),
                                               ]),
                      (get_param, reportflow, [('TR', 'ica_plot.TR')]),
                      (mainflow, reportflow, [('filterflow.masks_for_warp.mask_conf',
                                               'carpet_plot.brainmask')]),
                      (reportflow, datasink, [
                          ('compcor_plot.out_file', 'preproc_func.@compcor_plot'),
                          ('carpet_plot.out_file', 'preproc_func.@carpet_plot'),
                          ('confound_inspection.outlier_file', 'preproc_func.@conf_inspect'),
                          ('confound_inspection.plot_main', 'preproc_func.@conf_main'),
                          ('confound_inspection.plot_motion', 'preproc_func.@conf_motion'),
                          ('confound_inspection.plot_compA', 'preproc_func.@conf_compA'),
                          ('confound_inspection.plot_compT', 'preproc_func.@conf_compT'),
                          ('ica_plot.fig_signal', 'preproc_func.@fig_signal'),
                          ('ica_plot.fig_brain', 'preproc_func.@fig_brain'),
                      ]),
                     ])

In [ ]:
# Connect main and confound workflow with report workflow
preproc_func.connect([(mainflow, reportflow, [
                          ('filterflow.temporal_filter.mean_file', 'compcor_plot.mean'),
                          ('filterflow.temporal_filter.mean_file', 'ica_plot.mean_file'),
                          ('filterflow.masks_for_warp.mask_conf', 'compcor_plot.brainmask'),
                          ('filterflow.temporal_filter.out_file', 'carpet_plot.in_file'),
                          ]),
                      (confflow, reportflow, [
                          ('tCompCor.high_variance_masks', 'compcor_plot.maskT'),
                          ('acomp_masks.out_file', 'compcor_plot.maskA'),
                          ('combine_confounds.out_file', 'confound_inspection.confounds'),
                          ('compute_ica.comp_signal', 'ica_plot.comp_signal'),
                          ('compute_ica.comp_file', 'ica_plot.comp_file'),
                          ])
                     ])

## Visualize Workflow

In [ ]:
# Create preproc_func output graph
preproc_func.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph in the notebook (NBVAL_SKIP)
from IPython.display import Image
Image(filename=opj(preproc_func.base_dir, 'preproc_func', 'graph.png'))

# Run Workflow

In [ ]:
# Run the workflow in parallel mode
res = preproc_func.run(plugin='MultiProc', plugin_args={'n_procs' : n_proc})

In [ ]:
# Save workflow graph visualizations in datasink
preproc_func.write_graph(graph2use='flat', format='png', simple_form=True)
preproc_func.write_graph(graph2use='colored', format='png', simple_form=True)

from shutil import copyfile
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph.png'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph.png'))
copyfile(opj(preproc_func.base_dir, 'preproc_func', 'graph_detailed.png'),
         opj(exp_dir, out_dir, 'preproc_func', 'graph_detailed.png'));